### Практическая работа №2 по дисциплине "Математическое моделирование в САПР"
Выполнена студентом группы БИВТ-18-3 ИТКН НИТУ "МИСиС" Ковешниковым Арсением
#### Построение дерева достижимости сети Петри

In [11]:
import math
import numpy as np
import plotly as pl
import pandas as pd
import xlrd

Получим векторное описание сети Петри из .xlsx файла. Количество позиций определяется по количеству столбцов, переходов — по количеству входных/выходных векторов. Несовпадение количества входных/выходных векторов вызывает ошибку.

In [12]:
with open('petrinet.xlsx', 'rb') as file:
    df = pd.read_excel(file, engine='openpyxl', index_col=0)
    if (df.loc['I'].shape[0] != df.loc['O'].shape[0]):
        raise SyntaxError("Неверное содержание файла!")
display(df)

,p1,p2,p3,p4
m,1,0,1,0
I,1,1,0,0
I,0,0,0,1
I,0,0,1,0
O,1,0,0,0
O,0,1,1,0
O,0,0,0,1


Выведем количества позиций и переходов, начальную маркировку и входные/выходные вектора.

In [13]:
columns = df.shape[1]
pQty = df.shape[1]
tQty = df.loc['I'].shape[0]
init_m = df.loc['m'].array.to_numpy()
init_i = df.loc['I']
init_o = df.loc['O']
print("Кол-во позиций: "+str(pQty))
print("Кол-во переходов: "+str(tQty))
print("Начальная маркировка: "+str(init_m))
display(init_i)
display(init_o)

Кол-во позиций: 4
Кол-во переходов: 3
Начальная маркировка: [1 0 1 0]


,p1,p2,p3,p4
I,1,1,0,0
I,0,0,0,1
I,0,0,1,0


,p1,p2,p3,p4
O,1,0,0,0
O,0,1,1,0
O,0,0,0,1


Создадим класс для хранения данных об узле дерева достижимости

In [6]:
class Node:
    def __init__(self, parent=[], children=[], marking=None, type=0):
        self.parent = parent
        self.children = children
        self.marking = marking
        self.type = type
        
    def find(self, marking):
        if self.marking is marking: return True
        for node in self.children:
            n = node.find(x)
            if n.marking is not marking: return True
        return False
        
tree = Node(marking=init_m)

Реализуем функцию поиска разрешённых переходов. Если хотя бы один элемент входного вектора превышает маркировку, переход запрещён.

In [7]:
def allowedTransitions(node):
    result = []
    for i in range(tQty):
        allowed = True
        for j in range(pQty):
            if(node.marking[j]<init_i.iloc[i][j]):
                allowed = False
        if(allowed == True):
            result.append(i)
    return result

init_allowedTransitions = allowedTransitions(tree)

print("Изначально разрешённые переходы: ", [x+1 for x in init_allowedTransitions])

Изначально разрешённые переходы:  [3]


In [8]:
def performTransition(tNumber,fromNode):
    marking = []
    for i in range(pQty):
        marking.append(fromNode.marking[i]-init_i.iloc[tNumber][i]+init_o.iloc[tNumber][i])
    return marking

print(performTransition(2,tree))

[1, 0, 0, 1]


In [10]:
def isTerminal(node,tree):
    result = False
    if(allowedTransitions(node) == []):
        result = True
    return result
    
def isDuplicate(node,tree):
    result = False
    if(allowedTransitions(node) == []):
        result = True
    return result

def nodeType(node,tree):
    result = 0
    if(allowedTransitions(node) == []):
        node.type = 1
    

In [14]:
def execute(node):
    transitions = allowedTransitions(node)
    for t in transitions:
        marking = performTransition(t,node)
        newNode = Node(parent=node, marking=marking)
        node.children.append(newNode)
        #execute(newNode)
        
execute(tree)